In [1]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library sqlite3

In [2]:
con = sqlite3.connect('../data/checking-logs.sqlite')

In [11]:
pd.read_sql('SELECT * FROM pageviews LIMIT 10', con)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


In [12]:
pd.read_sql('SELECT * FROM checker LIMIT 10', con)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


##  Create a new table `datamart` in the database by `joining` the tables `pageviews` and `checker` using only one query

- the table should have the following columns: `uid`, `labname`, `first_commit_ts`, `first_view_ts`
- `first_commit_ts` is just a new name of the column `timestamp` from the `checker` table, it shows the first commit from a particular lab and from a particular user
- `first_view_ts` is the first visit of a user to the table `pageviews`, timestamp when a user visited the newsfeed
- `status = ’ready’` should still be a filter
- `numTrials = 1` should still be a filter
- `labname`s should still be from the list: `’laba04’`, `’laba04s’`, `’laba05’`, `’laba06’`, `’laba06s’`, `’project1’`
- the table should contain only the users (`uids` with `user_`*) and not the `admin`s
- `first_commit_ts` and `first_view_ts` should be parsed as `datetime64`

In [44]:
cur = con.cursor()
cur.execute('DROP TABLE IF EXISTS datamart')

query = """
CREATE TABLE datamart AS SELECT
    a.uid, a.labname, a.timestamp AS first_commit_ts, b.datetime AS first_view_ts
FROM (
    SELECT
        uid, labname, timestamp
    FROM
        checker
    WHERE
        uid LIKE 'user_%'
        AND
        status = 'ready'
        AND
        numTrials = 1
        AND
        labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
) a
LEFT JOIN (
    SELECT
        uid, MIN(datetime) as datetime
    FROM
        pageviews
    GROUP BY
        uid
) b
ON
    a.uid = b.uid
"""
cur.execute(query)

datamart = pd.read_sql('SELECT * FROM datamart', con, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


In [25]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


## Create two dataframes: `test` and `control`

- `test` should have the ***users that have the values*** in `first_view_ts`
- `control` should have the ***users that have missing values*** in `first_view_ts`
- replace the missing values in the `control` with the ***average `first_view_ts`*** of the test users, we will use this value for the future analysis
- save both tables into the database, you will use them in the next exercises

In [68]:
test = pd.read_sql("SELECT * FROM datamart WHERE IFNULL(first_view_ts, 0)",
                   con, parse_dates=['first_commit_ts', 'first_view_ts'])
test.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


In [75]:
control = pd.read_sql("SELECT * FROM datamart WHERE NOT IFNULL(first_view_ts, 0)",
                      con, parse_dates=['first_commit_ts', 'first_view_ts'])
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.7+ KB



---

### replace the missing values in the control with the average first_view_ts of the test users

In [76]:
control['first_view_ts'] = test['first_view_ts'].mean()
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.7+ KB



---

### save both tables into the database, you will use them in the next exercises

In [74]:
test.to_sql('test', con, if_exists='replace')
control.to_sql('control', con, if_exists='replace')

81

In [78]:
pd.read_sql('SELECT * FROM test', con).head()

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


In [79]:
pd.read_sql('SELECT * FROM control', con).head()

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783
1,1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783
2,2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783
3,3,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783
4,4,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783



---

### close the connection

In [80]:
con.close()